# Libraries

In [1]:
# LIBRARIES #

import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot, CustomJS, ColumnDataSource, Rect
from bokeh.layouts import row, gridplot, column
from bokeh.models.widgets import Slider, Button
from bokeh.events import ButtonClick

import SimpleITK as sitk 

from Gafchromic_multichannel import GafchromicFilms

import time


output_notebook()

Loading BokehJS ...

In [2]:
import warnings

ignoreW = True
if ignoreW :
    warnings.filterwarnings('ignore')

# Functions

In [3]:
# CONVERTS THE IMG TO RGBA FOR DISPLAY #

def convertToRGBA(array, sizex, sizey):

    # creates a new rgba img and copy the tiff values in it
    rgba = np.empty((sizey,sizex), dtype=np.uint32)
    view = rgba.view(dtype=np.uint8).reshape((sizey, sizex, 4))
    view[:,:,0] = array[:,:,0]/65535.0*255.0
    view[:,:,1] = array[:,:,1]/65535.0*255.0
    view[:,:,2] = array[:,:,2]/65535.0*255.0
    view[:,:,3] = 255

    return rgba

In [4]:
# PLOTS IMAGE AND ONE X PROFILE #
# @params:
#   img: array img to display
#   sizex: size of the img in x
#   sizey: size of the img in y
#   linePos: position of the line profile
#   dSum: lines to use for the mean profile are the linePos +- dSum
#   imgPlotWidth: width of the image plot

def simpleImageDisplay(img, sizex, sizey, linePos, dSum=1, imgPlotWidth=500, LinePlotHeight = 300, minvalue=0, maxv=0):

    
    # Main image dose display:
    if (maxv==0): maxvalue = np.amax(img)
    else: maxvalue = maxv

    color_mapper = LinearColorMapper(palette="Viridis256", low=minvalue, high=maxvalue)
    
    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                    label_standoff=12, border_line_color=None, location=(0,0),
                    title='')
    
    p1 = figure(plot_width=int(imgPlotWidth*1.1), plot_height=int(imgPlotWidth*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Dose image", toolbar_location="above")
    
    p1.image(image=[img], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper)
    p1.line(x=[0,sizex-1], y=[linePos, linePos], line_width=2, line_color="white")
    p1.rect(x=int(sizex/2), y=linePos, width=sizex, height=dSum*2+1, fill_color="white", alpha=0.2) 
    
    p1.add_layout( color_bar, 'right')
    
    # Line profile display:
    yLineDose = np.sum(img[linePos-dSum:linePos+dSum+1,:]/(2*dSum+1), axis=0)
    maxvalue = np.amax(yLineDose)
    minvalue = np.amin(yLineDose)
    
    p2 = figure(plot_width=int(imgPlotWidth*1.1), plot_height=int(LinePlotHeight), 
                    x_range=(0,sizex), y_range=(minvalue,maxvalue), 
                    title="Horizontal dose profile", toolbar_location="above")
    
    p2.line(x=range(sizex), y=yLineDose, line_color="#2690d4", line_width=3, line_alpha=1.0)
    
    # Grid:
    grid = gridplot([[p1],[p2]])
    show(grid)

In [5]:
# PLOTS 2 IMAGEs AND ONE X PROFILE FOR THE 2 IMAGES #
# Images must be of the same size
# @params:
#   img1: array img to display
#   img2: array img to display
#   sizex: size of the img in x
#   sizey: size of the img in y
#   linePos: position of the line profile
#   dSum: lines to use for the mean profile are the linePos +- dSum
#   imgPlotWidth: width of the image plot

def doubleImageDisplay(img1, img2, sizex, sizey, linePos, dSum=1, imgPlotWidth=700, LinePlotHeight = 300, identicalScale = True):


    # max dose:
    maxdose1 = np.amax(img1)
    maxdose2 = np.amax(img2)
    if identicalScale: 
        maxdose1 = maxdose2 = max(maxdose1, maxdose2)
    
    # First image display
    color_mapper1 = LinearColorMapper(palette="Viridis256", low=0, high=maxdose1)
    
    color_bar1 = ColorBar(color_mapper=color_mapper1, ticker=BasicTicker(),
                    label_standoff=12, border_line_color=None, location=(0,0),
                    title='Dose cGy')
    
    p1 = figure(plot_width=int(imgPlotWidth/2*1.1), plot_height=int(imgPlotWidth/2*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Dose image 1", toolbar_location="above")
    
    p1.image(image=[img1], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper1)
    p1.line(x=[0,sizex-1], y=[linePos, linePos], line_width=2, line_color="white")
    p1.rect(x=int(sizex/2), y=linePos, width=sizex, height=dSum*2+1, fill_color="white", alpha=0.2) 

    
    # Second image display
    color_mapper2 = LinearColorMapper(palette="Viridis256", low=0, high=maxdose2)
    
    color_bar2 = ColorBar(color_mapper=color_mapper2, ticker=BasicTicker(),
                    label_standoff=12, border_line_color=None, location=(0,0),
                    title='Dose cGy')

    p2 = figure(plot_width=int(imgPlotWidth/2*1.1), plot_height=int(imgPlotWidth/2*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Dose image 2", toolbar_location="above")
    
    p2.image(image=[img2], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper2)
    p2.line(x=[0,sizex-1], y=[linePos, linePos], line_width=2, line_color="white")
    p2.rect(x=int(sizex/2), y=linePos, width=sizex, height=dSum*2+1, fill_color="white", alpha=0.2) 
    
    
    # Line profiles display:
    yLineDose1 = np.sum(img1[linePos-dSum:linePos+dSum,:]/(2*dSum+1), axis=0)
    maxdose1 = int(np.amax(yLineDose1))
    
    yLineDose2 = np.sum(img2[linePos-dSum:linePos+dSum,:]/(2*dSum+1), axis=0)
    maxdose2 = int(np.amax(yLineDose2))

    p3 = figure(plot_width=int(imgPlotWidth*1.1), plot_height=int(LinePlotHeight), 
                    x_range=(0,sizex), y_range=(0,int(max(maxdose1, maxdose2))+1), 
                    title="Horizontal dose profiles", toolbar_location="above")
    
    p3.line(range(sizex), yLineDose1, line_color="firebrick", line_width=3, legend='Img 1')
    p3.line(range(sizex), yLineDose2, line_color="blue", line_width=3, legend='Img 2')
    
    
    # Grid:
    grid = gridplot([[p1,p2],[p3]])
    show(grid)
    
    

In [6]:
# PLOTS THE GAFCHROMIC IMAGE AND THE RECTANGLES USED FOR CALIBRATION #
# @params:
#   rgbaimg: array img to display
#   sizex: size of the img in x
#   sizey: size of the img in y
#   rectlist: list of rectangles used for calibration
#   dimviewer: width of the image plot

def displayRectanglesOnImage(filename, rectlist, dimviewer) :
    img = sitk.ReadImage(filename)
    sizex = img.GetWidth()
    sizey = img.GetHeight()
    array = sitk.GetArrayFromImage(img)
    rgbaimg = convertToRGBA(array, sizex, sizey)
    
    p = figure(plot_width=int(dimviewer*0.95), plot_height=int(dimviewer*0.95*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Gafchromic image", toolbar_location="above")
    
    p.image_rgba(image=[rgbaimg], x=[0], y=[0], dw=[sizex], dh=[sizey])

    for i in range(len(rectlist)):
        p.rect(x=int(rectlist[i][0]+(rectlist[i][2]-rectlist[i][0])/2), 
               y=int(rectlist[i][1]+(rectlist[i][3]-rectlist[i][1])/2), 
               width=rectlist[i][2]-rectlist[i][0],
               height=rectlist[i][3]-rectlist[i][1], 
               fill_color="#0096ff", 
               alpha=0.2)

    show(p)

# Calibration curve:

In [7]:
# INPUT PARAMETERS:
# <!> ne pas mettre d'accent dans les chemins et noms de fichiers


m_path = 'etalonnage 24h/'
m_nbOfFiles = 1
m_firstNb = 1
m_filesName = "etalonnage_24h_000"
m_fileExtension = ".tif"

m_dimViewer = 600


# Liste de dose (il est nécessaire de classer les dose dans un ordre croissant ou décroissant)
m_dose = [0, 25, 50, 100, 200, 300, 400, 500, 600, 800]


# Rectangles de l'image correspondant aux doses énoncées au dessus
rectList = []
rectList.append([1200,700,2000,900])
rectList.append([1200,1000,2000,1050])
rectList.append([1200,1300,2000,1500])
rectList.append([1200,1600,2000,1800])
rectList.append([1200,1900,2000,2100])
rectList.append([1200,2200,2000,2400])
rectList.append([1200,2500,2000,2700])
rectList.append([1200,2800,2000,3000])
rectList.append([1200,3110,2000,3310])
rectList.append([1200,3420,2000,3620])


#displayRectanglesOnImage(m_path+m_filesName+str(m_firstNb)+m_fileExtension, rectList, m_dimViewer)

print("dONE !")

dONE !


In [8]:


# Reads the images and computes the mean image:
try:
    g = GafchromicFilms(m_path+m_filesName, m_firstNb, m_nbOfFiles)
    #g.applyWienerFilter()
    #g.applyStreakCorrection()
    g.multilinearRegression(m_dose, rectList, dispResults=True)
    #g.saveMultilinearRegressionFile(m_path+'multiLinearRegressionCoefficients_02282001.txt', 
    #                        batchNb='02282001', dispStatus=False)
    rgbimg = g.getArray()
    size = g.getSize()
    print("dONE !")
except ValueError as err:
    print('Erreur: ' + err)


Calculated coefficients:
[ -620.34840552  3155.0213648  -4357.83146908]


dONE !


In [ ]:
#simpleImageDisplay(rgbimg[800:3800,850:2350,0], 1500, 3000, 1500, dSum=0)

# Convert Gafchromic films to dose:

In [ ]:
# INPUT PARAMETERS:
# <!> ne pas mettre d'accent dans les chemins et noms de fichiers


#m_path = 'X:/aria/RTPlans/rom50bis/'
#m_nbOfFiles = 5
#m_firstNb = 1
#m_filesName = "RC_0"
#m_fileExtension = ".tif"

#m_path = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/test IA/25h/'
#m_nbOfFiles = 1
#m_firstNb = 1
#m_filesName = "ordreA 25h acq_no0"
#m_fileExtension = ".tif"


m_path = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/'
m_nbOfFiles = 2
m_firstNb = 1
m_filesName = "etalonnage_24h_000"
m_fileExtension = ".tif"


m_calibrationFile = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/multiLinearRegressionCoefficients_02282001.txt'

m_dimViewer = 600



# Rectangles de l'image correspondant aux doses énoncées au dessus
#ctrlRect = [150,200,470,230]
#toDoseRect = [100,240,520,760]
ctrlRect = [1100,700,2000,900]
toDoseRect = [800,950,2300,3700]

rectList = []
rectList.append(ctrlRect)
rectList.append(toDoseRect)

displayRectanglesOnImage(m_path+m_filesName+str(m_firstNb)+m_fileExtension, rectList, m_dimViewer)

In [ ]:
try:
    subsampleF = 1
    
    g = GafchromicFilms(m_path+m_filesName, m_firstNb, m_nbOfFiles)
    #g.applyStreakCorrection()
    #g.subSampleDataArray(subsampleF)
    #g.applyWienerFilter()

    g.readMultilinearRegressionFile(m_calibrationFile, dispStatus=False)
    
    ctrlRect_sub = [int(ctrlRect[0]/subsampleF), int(ctrlRect[1]/subsampleF), \
                    int(ctrlRect[2]/subsampleF), int(ctrlRect[3]/subsampleF)]
    toDoseRect_sub = [int(toDoseRect[0]/subsampleF), int(toDoseRect[1]/subsampleF), \
                    int(toDoseRect[2]/subsampleF), int(toDoseRect[3]/subsampleF)]
    
    #doseimg = g.convertToDose(toDoseRect_sub, ctrlRect_sub)
    doseimg = g.convertToDoseWithFingerPrint(toDoseRect_sub, ctrlRect_sub, dispStatus=True, convThreashold=0.0001)
    
    rgbimg = g.getArray()
    size = g.getSize()
    print("dONE !")
except ValueError as err:
    print('Erreur: ' + err)


In [ ]:
try:
    g.saveToTiff(doseimg, m_path+'imgConvertedToDose-multilinearWithFingerPrint.tif')
except ValueError as err:
    print('Erreur: ' + err)


In [ ]:
# Simple Viewer: (can be used to create mean profiles)
#simpleImageDisplay(rgbimg[:,:,0], rgbimg.shape[1], rgbimg.shape[0], int(rgbimg.shape[0]/2), dSum=0)
#simpleImageDisplay(rgbimg[1850:2750,550:2700,0], 2150, 900, 450, dSum=0)
simpleImageDisplay(doseimg, doseimg.shape[1], doseimg.shape[0], int(doseimg.shape[0]/3), dSum=1)
